In [1]:
import pandas as pd

In [2]:
aisles_df = pd.read_csv('aisles.csv')
departments_df = pd.read_csv('departments.csv')
ord_prod_prior_df = pd.read_csv('order_products__prior.csv')
ord_prod_train_df = pd.read_csv('order_products__train.csv')
orders_df = pd.read_csv('orders.csv')
products_df = pd.read_csv('products.csv')

In [3]:
new_prod = pd.merge(products_df, aisles_df, on='aisle_id')
new_prod.head()

,product_id,product_name,aisle_id,department_id,aisle
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes
2,102,Danish Butter Cookies,61,19,cookies cakes
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes


In [4]:
new_prod = pd.merge(new_prod, departments_df, on='department_id')
new_prod.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks


In [5]:
new_prod.to_csv('Prods.csv', index=False)

In [6]:
orders = orders_df.drop(columns=['eval_set'])
orders.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [7]:
orders.to_csv('Ord.csv', index=False)

In [8]:
ord_prod = pd.concat([ord_prod_train_df, ord_prod_prior_df])
ord_prod.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [9]:
ord_prod.to_csv('Ord_Prod.csv', index=False)

Isolate for add_to_cart_order = 1, count iterations

In [10]:
mvp_df = ord_prod.loc[ord_prod['add_to_cart_order']==1]
mvp_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
8,36,39612,1,0
16,38,11913,1,0
25,96,20574,1,1
32,98,8859,1,1


In [11]:
mvp_df = mvp_df.groupby(by='product_id').sum()
mvp_df = mvp_df.sort_values('add_to_cart_order', ascending=False)
mvp_df = mvp_df.drop(columns=['order_id'])
mvp_df.head(5)

,add_to_cart_order,reordered
product_id,,
24852,115521,102269
13176,82877,72651
27845,32071,27781
21137,28875,23022
47209,24913,20389


In [17]:
mvp_df.to_csv('MVP.csv', index=True)

Count product_id w/ reorder = 0

In [13]:
lvp_df = ord_prod.loc[ord_prod['reordered']==0]
lvp_df = lvp_df.replace(0,1)
lvp_df = lvp_df.groupby(by='product_id').sum()
lvp_df = lvp_df.sort_values('reordered', ascending = False)
lvp_df = lvp_df.rename(columns={'reordered': 'not_reordered'})
lvp_df.head(30)

,order_id,add_to_cart_order,not_reordered
product_id,,,
24852,129691900667,445841,76125
13176,112357770966,404040,65655
21137,104451979227,472740,61129
21903,97411525764,454447,56766
47626,83225167687,422162,48614
26209,79751486732,429491,46658
16797,76798991645,361447,44857
47209,76187500386,339794,44704
47766,75122446758,321086,43954


In [14]:
lvp_df.to_csv('LVP.csv', index=True)

Mean of ordered items

In [15]:
item_order = ord_prod
item_order['product_id'] = 1
item_order = item_order.groupby('order_id').sum()
item_order = item_order.drop(columns=['add_to_cart_order'])
item_order = item_order.rename(columns={'product_id': 'nb_items'})
item_order.head()

,nb_items,reordered
order_id,,
1,8,4
2,9,6
3,8,8
4,13,12
5,26,21


In [18]:
item_order.to_csv('Item_Order.csv', index=True)

Calculate number of products by order